In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sqlalchemy import create_engine, text
!pip install PyMySql
import pymysql
import sqlalchemy

!pip install ipython-sql

#Import modules that related to SQL and connection to Azure

#Create the engine so that we can get access to Azure MySQL
username = 'A12' # Your team name
password = 'A12password'
host = 'testproject.mysql.database.azure.com'
database = 'A12'  # Your team name

connection_string = f"mysql+pymysql://{username}:{password}@{host}/{database}"

ssl_args = {
    'ssl_ca': 'path_to/ca-cert.pem',
    'ssl_cert': 'path_to/client-cert.pem',
    'ssl_key': 'path_to/client-key.pem'
}

engine = create_engine(
    connection_string,
    connect_args={'ssl': ssl_args}
)


I tried to install **mysqlclient** package as well, but the installation will fail for some reason. So I have to use **pymysql** for the following import data and query process.

In [ ]:
data_file = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json")
data = []
for line in data_file:
  data.append(json.loads(line))
checkin_df = pd.DataFrame(data)
data_file.close()

The DF checkin's date column is too long for SQL, so going to split it to multiple rows

In [ ]:
checkin_df['date'] = checkin_df['date'].str.split(',')
checkin_df_exploded = checkin_df.explode('date')
print(checkin_df_exploded)

In [ ]:
data_file = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json")
data = []
for line in data_file:
  data.append(json.loads(line))
business_df = pd.DataFrame(data)
data_file.close()

In [ ]:
data_file = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json")
data = []
for line in data_file:
  data.append(json.loads(line))
review_df = pd.DataFrame(data)
data_file.close()

In [ ]:
data_file = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json")
data = []
for line in data_file:
  data.append(json.loads(line))
tip_df = pd.DataFrame(data)
data_file.close()

In [ ]:
data_file = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json")
data = []
for line in data_file:
  data.append(json.loads(line))
user_df = pd.DataFrame(data)
data_file.close()

In [ ]:
#Check if all the dataset has already been imported as DataFrames
print(type(business_df))
print(type(user_df))
print(type(checkin_df_exploded))
print(type(review_df))
print(type(tip_df))

Set the authorization settings for Azure, called engine.

In [ ]:
business_df.to_sql('business', engine, if_exists='replace', index=True, 
                   dtype={'attributes': sqlalchemy.types.JSON, 'hours': sqlalchemy.types.JSON}, chunksize = 50000)

When importing checkin table to Azure, an error occurs says that the column of date is too long. Considering that the 'date' column is a huge amount of dates, in order to import successfully, Cut off the date and set a upper limit of 255 characters

In [ ]:
max_length = 255 #Set the maximum length for every element in 'date' column
checkin_df['date'] = checkin_df['date'].apply(lambda x: x[:max_length] if isinstance(x, str) else x) #Cut off all the characters that is over 255 limit
user_df['friends'] = user_df['friends'].apply(lambda x: x[:max_length] if isinstance(x, str) else x) 

In [ ]:
checkin_df_exploded.to_sql('checkin', engine, if_exists='replace', index = True, chunksize = 50000)

In [ ]:
review_df.iloc[:1000].to_sql('review', engine, if_exists='replace', index=True, chunksize = 50000)

In [ ]:
tip_df.to_sql('tip', engine, if_exists='replace', index=True, chunksize = 50000)

In [ ]:
user_df.to_sql('user', engine, if_exists='replace', index=True, chunksize = 50000)

After importing all the tables to Azure, we need to learn the attributes of these tables: **business, checkin, review, tip & user**

In [ ]:
#The query is used to learn the column names and other related information for the five tables
query = "DESCRIBE business;"
description_df = pd.read_sql_query(text(query), engine)
print(description_df)

Following code is used for querying the corresponding data from SQL entities: **business, checkin, tip, user & review.**

In [ ]:
# Define your SQL query as a string
query1 = "SELECT * FROM a12.business where state = 'IL';"

# Use the engine to execute the query and load the result into a DataFrame
result_df1 = pd.read_sql_query(text(query), engine)

# Now result_df contains the first five rows of your_table_name
result_df1

In [ ]:
# Define your SQL query as a string
query2 = "SELECT * FROM a12.business where stars > 4;"

# Use the engine to execute the query and load the result into a DataFrame
result_df2 = pd.read_sql_query(text(query), engine)

# Now result_df contains the first five rows of your_table_name
result_df2

In [ ]:
# Define your SQL query as a string
query3 = "SELECT * FROM a12.business where stars =5 AND state='IL';"

# Use the engine to execute the query and load the result into a DataFrame
result_df3 = pd.read_sql_query(text(query), engine)

# Now result_df contains the first five rows of your_table_name
result_df3

In [ ]:
# Define your SQL query as a string
query4 = "SELECT name, categories, review_count FROM a12.business WHERE city like 'New Orleans'ORDER BY review_count DESC LIMIT 10;"

# Use the engine to execute the query and load the result into a DataFrame
result_df4 = pd.read_sql_query(text(query), engine)

# Now result_df contains the first five rows of your_table_name
result_df4

In [ ]:
# Define your SQL query as a string
query5 = "SELECT DISTINCT categories FROM a12.business where stars =5 AND state='IL';"

# Use the engine to execute the query and load the result into a DataFrame
result_df5 = pd.read_sql_query(text(query), engine)

# Now result_df contains the first five rows of your_table_name
result_df5

In [ ]:
# Define your SQL query as a string
query6 = "SELECT name, stars FROM a12.business WHERE city = 'New Orleans' AND review_count > 100 ORDER BY stars DESC LIMIT 5;;"

# Use the engine to execute the query and load the result into a DataFrame
result_df6 = pd.read_sql_query(text(query), engine)

# Now result_df contains the first five rows of your_table_name
result_df6

In [ ]:
# Define your SQL query as a string
query7 = "SELECT categories, COUNT(*) AS count FROM a.12business WHERE city = 'New Orleans' AND stars > 4.5 GROUP BY categories ORDER BY count DESC LIMIT 5;"

# Use the engine to execute the query and load the result into a DataFrame
result_df7 = pd.read_sql_query(text(query), engine)

# Now result_df contains the first five rows of your_table_name
result_df7

In [ ]:
# Define your SQL query as a string
query8 = "SELECT categories, AVG(review_count) AS avg_review_count FROM a12.business WHERE city = 'New Orleans' GROUP BY categories ORDER BY avg_review_count DESC;"

# Use the engine to execute the query and load the result into a DataFrame
result_df8 = pd.read_sql_query(text(query), engine)

# Now result_df contains the first five rows of your_table_name
result_df8

In [ ]:
# Define your SQL query as a string
query9 = "SELECT b.name, AVG(r.stars) AS avg_stars, COUNT(r.review_id) AS number_of_reviews FROM a12.business b JOIN review r ON b.business_id = r.business_id WHERE b.city = 'New Orleans'GROUP BY b.name;"

# Use the engine to execute the query and load the result into a DataFrame
result_df9 = pd.read_sql_query(text(query), engine)

# Now result_df contains the first five rows of your_table_name
result_df9

In [ ]:
# Define your SQL query as a string
query10 = "SELECT b.categories AS category, COUNT(chk.business_id) AS total_checkins FROM a12.business b JOIN checkin chk ON b.business_id = chk.business_id WHERE b.city = 'New Orleans' GROUP BY b.categories ORDER BY total_checkins DESC LIMIT 5;"

# Use the engine to execute the query and load the result into a DataFrame
result_df10 = pd.read_sql_query(text(query), engine)

# Now result_df contains the first five rows of your_table_name
result_df10

**In case anyone need to delete a table, please use the following code, otherwise, don't change the code and run it:**

In [ ]:
from sqlalchemy import inspect
from sqlalchemy import MetaData, Table

def table_exists(name, con):
    inspector = inspect(con)
    return name in inspector.get_table_names()

metadata = MetaData()

# Drop the table if it exists. Please replace the 'table_name' to the target table name
if table_exists('table_name', engine):
    table = Table('table_name', metadata)
    table.drop(engine)